# 엘모

## 09-09 엘모(Embeddings from Language Model, ELMo)

- 사전 훈련된 언어 모델(Pre-trained language model) 사용

### 1. ELMo(Embeddings from Language Model)
- 같은 표기의 단어라도 문맥에 따라서 다르게 워드 임베딩을 할 수 있으면 자연어 처리의 성능을 올릴 수 있음
- 워드 임베딩 시 문맥을 고려해서 임베딩을 하겠다는 아이디어가 문맥을 반영한 워드 임베딩(Contextualized Word Embedding)

### 2. biLM(Bidirectional Language Model)의 사전 훈련
- 은닉층이 2개인 일반적인 단방향 RNN 언어 모델의 언어 모델링

  ![img](https://wikidocs.net/images/page/33930/deepbilm.PNG)
  - RNN 내부의 은닉 상태 $h_t$는 시점(time step)이 지날수록 점점 업데이트
  - RNN의 $h_t$의 값이 문장의 문맥 정보를 점차적으로 반영

- ELMo는 위의 그림의 순방향 RNN 뿐만 아니라, 위의 그림과는 반대 방향으로 문장을 스캔하는 역방향 RNN 또한 활용
- ELMo는 양쪽 방향의 언어 모델을 둘 다 학습하여 활용한다고하여 이 언어 모델을 biLM(Bidirectional Language Model) 이라고 함
- ELMo에서 말하는 biLM은 기본적으로 다층 구조(Multi-layer)를 전제

- 은닉층이 2개인 순방향 언어 모델과 역방향 언어 모델의 모습
  ![img](https://wikidocs.net/images/page/33930/forwardbackwordlm2.PNG)
  - biLM의 각 시점의 입력이 되는 단어 벡터는 합성곱 신경망을 이용한 문자 임베딩(character embedding)을 통해 얻은 단어 벡터
  - 문자 임베딩은 문맥과 상관없이 dog란 단어와 doggy란 단어의 연관성을 찾아낼 수 있음
  - 문자 임베딩은 OOV에도 견고

- 양방향 RNN != ELMo에서의 biLM
  - 양방향 RNN은 순방향 RNN의 은닉 상태와 역방향의 RNN의 은닉 상태를 연결(concatenate)하여 다음층의 입력으로 사용
  - biLM의 순방향 언어모델과 역방향 언어모델이라는 두 개의 언어 모델을 별개의 모델로 보고 학습

### 3. biLM의 활용
- biLM이 언어 모델링을 통해 학습된 후 ELMo가 사전 훈련된 biLM을 통해 입력 문장으로부터 단어를 임베딩하기 위한 과정

  ![img](https://wikidocs.net/images/page/33930/playwordvector.PNG)
 
  - play란 단어가 임베딩이 되고 있다는 가정 하에 ELMo를 설명
  - play라는 단어를 임베딩 하기위해 ELMo는 위의 점선의 사각형 내부의 각 층의 결과값을 재료로 사용
  - 다시 말해 해당 시점(time step)의 BiLM의 각 층의 출력값을 가져옴
  - 순방향 언어 모델과 역방향 언어 모델의 각 층의 출력값을 연결(concatenate)하고 추가 작업을 진행

- ELMo가 임베딩 벡터를 얻는 과정
  #### (1) 각 층의 출력값을 연결(concatenate)한다.
  ![img](https://wikidocs.net/images/page/33930/concatenate.PNG)

  #### (2) 각 층의 출력값 별로 가중치를 준다.
  ![img](https://wikidocs.net/images/page/33930/weight.PNG)

  #### (3) 각 층의 출력값을 모두 더한다.
  ![img](https://wikidocs.net/images/page/33930/weightedsum.PNG)
  - (2)번과 (3)번의 단계를 요약하여 가중합(Weighted Sum)을 한다고 할 수 있음

  #### (4) 벡터의 크기를 결정하는 스칼라 매개변수를 곱한다.
  ![img](https://wikidocs.net/images/page/33930/scalarparameter.PNG)

- 이렇게 완성된 벡터를 ELMo 표현(representation)이라고 함
- ELMo를 입력으로 사용하고 수행가능한 것: 텍스트 분류, 질의 응답 시스템 등의 자연어 처리 작업

- ELMo 표현을 어떻게 텍스트 분류 작업에 사용하는 방법
  - ELMo 표현을 기존의 임베딩 벡터와 함께 사용
  - 텍스트 분류 작업을 위해서 GloVe와 같은 기존의 방법론을 사용한 임베딩 벡터를 준비했다고 가정
  - 준비된 ELMo 표현을 GloVe 임베딩 벡터와 연결(concatenate)해서 입력으로 사용
  -  biLM의 가중치는 고정시키고, 위에서 사용한 $s_1,s_2,s_3$ 과 $γ$는 훈련 과정에서 학습
  ![img](https://wikidocs.net/images/page/33930/elmorepresentation.PNG)


### 4. ELMo 표현을 사용해서 스팸 메일 분류하기


---




In [5]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
!pip install tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import tensorflow_hub as hub
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

In [8]:
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [9]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
# 필요한 v1과 v2만 사용
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
y_data = list(data['v1'])
X_data = list(data['v2'])

In [11]:
X_data[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [12]:
print(y_data[:5])

[0, 0, 1, 0, 0]


In [13]:
# 전체 데이터 개수의 80%와 20% 개수 확인
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train) # 80프로 개수
print(n_of_test) # 20프로 개수

5572
4457
1115


In [14]:
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

In [15]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

In [16]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text) # ELMo를 이용한 임베딩 층
hidden_layer = Dense(256, activation='relu')(embedding_layer) # 256개 뉴런이 있는 은닉층
output_layer = Dense(1, activation='sigmoid')(hidden_layer) # 1개의 뉴런을 통해 이진분류 수행
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # 손실함수 binary_crossentropy

In [17]:
history = model.fit(X_train, y_train, epochs=1, batch_size=60)

Train on 4457 samples
4457/4457 [==============================] - 57s 13ms/sample - loss: 0.1237 - acc: 0.9533


In [18]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates



 테스트 정확도: 0.9776
